Importing libraries

In [ ]:
!pip install -q torch
!pip install -q qdrant-client
!pip install -q sentence-transformers


In [14]:
import pandas as pd
import torch
import warnings
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.http import models
import pickle
from transformers import pipeline

Loading dataset and droping the column named index

In [7]:
CSV_path='/content/bigBasketProducts.csv'
df=pd.read_csv(CSV_path)
df=df.drop(columns = ["index"])

SentenceTransformer pre-trained models to convert our text into vectors.

In [8]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
# retriever = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
model

.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.8k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

Using sentence encoder model to compute embedding

In [9]:
class make_embedding_ds(torch.utils.data.IterableDataset):
    def __init__(self, csv = df):

        super(make_embedding_ds).__init__()
        self.csv = csv
        self.total_row = csv.shape[0]
        self.col = csv.columns.to_list()

    def __iter__(self):
        for row_no in range(self.total_row):

            row = self.csv.iloc[row_no].to_dict()
            product_name = row[self.col[0]]
            data = f"{row[ self.col[0] ]} is of category {row[self.col[1]]} and sub category is {row[self.col[2]]}. {row[self.col[0]]} is type {row[self.col[6]]}. brand of {row[self.col[0]]} is {row[self.col[3]]}, with rating {row[self.col[7]]}. sale price of {row[self.col[0]]} is {row[self.col[4]]} with market price {row[self.col[5]]}, description of {row[self.col[0]]} is {row[self.col[8]]}"
            vectors_emb = model.encode(data)


            yield row_no,product_name,data,vectors_emb


ds = make_embedding_ds(df)


dataloader = torch.utils.data.DataLoader(ds, num_workers=1,batch_size=128)


Creating a dictionary (json_embedding) which is storing embedding and metadata(payload) and creating a file which contains this vector representation

In [12]:
json_embedding = {
    "payload":[],
    "emb": []
}
batch_no = 1
for row,product_name,story,emb in dataloader:

    _batch_len = len(row)



    print(f"batch no-{batch_no} competed")
    batch_no += 1

    for index in range(_batch_len):

        temp = {}
        temp["id"] = row[index].item()
        temp["product"] = product_name[index]
        temp["story"] = story[index]

        json_embedding["payload"].append(temp)
        json_embedding["emb"].append(emb[index].numpy().tolist())




#     json_emb["payload"].append(batch[0])
#     json_emb["emb"].append(batch[1][0])


batch no-1 competed
batch no-2 competed
batch no-3 competed
batch no-4 competed
batch no-5 competed
batch no-6 competed
batch no-7 competed
batch no-8 competed


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7e7fce4ff1c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


batch no-9 competed
batch no-10 competed
batch no-11 competed
batch no-12 competed
batch no-13 competed
batch no-14 competed
batch no-15 competed
batch no-16 competed
batch no-17 competed
batch no-18 competed
batch no-19 competed
batch no-20 competed
batch no-21 competed
batch no-22 competed
batch no-23 competed
batch no-24 competed
batch no-25 competed
batch no-26 competed
batch no-27 competed
batch no-28 competed
batch no-29 competed
batch no-30 competed
batch no-31 competed
batch no-32 competed
batch no-33 competed
batch no-34 competed
batch no-35 competed
batch no-36 competed
batch no-37 competed
batch no-38 competed
batch no-39 competed
batch no-40 competed
batch no-41 competed
batch no-42 competed
batch no-43 competed
batch no-44 competed
batch no-45 competed
batch no-46 competed
batch no-47 competed
batch no-48 competed
batch no-49 competed
batch no-50 competed
batch no-51 competed
batch no-52 competed
batch no-53 competed
batch no-54 competed
batch no-55 competed
batch no-56 co

In [13]:
with open('Output1.pkl', 'wb') as file:
    pickle.dump(json_embedding, file)

In [27]:
dataloader

opening the output1.pkl file

In [15]:
Vec_PATH = "/content/Output1.pkl"
with open(Vec_PATH, 'rb') as file:
    vs = pickle.load(file)

Making collections

In [28]:
client = QdrantClient(":memory:")
collection_name = "BBProduct"

collections = client.get_collections()
if collection_name not in [c.name for c in collections.collections]:
    client.recreate_collection(
        collection_name=collection_name,
        vectors_config=models.VectorParams(
            size=384, #taking size as 384 as this is the size of the vectors in that collection. 384 is the output dimensionality of the encoder we are using.
            distance=models.Distance.COSINE,
        ),
    )
print(client.get_collections())

collections=[CollectionDescription(name='product')]


In [29]:
total_records = len(vs["payload"])
_payload = vs["payload"]
_emb = vs["emb"]
ids = list(range(0,total_records))

batch_size = 2

client.upsert(
    collection_name=collection_name,
    points=models.Batch(ids=ids, vectors=_emb, payloads=_payload),
)

collection_vector_count = client.get_collection(collection_name=collection_name).vectors_count
print(f"Vector count in collection: {collection_vector_count}")


Vector count in collection: 27555


In [30]:
with open('qs_client.pkl', 'wb') as file:
    pickle.dump(client, file)

Storing the sentence tranformation model in bermodel.pkl file

In [31]:
if torch.cuda.is_available(): #checking gpu is available or not
    device = "cuda"
else:
    device = "cpu"

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1", device=device)
model
with open('stmodel.pkl', 'wb') as file:
    pickle.dump(model, file)

Storing the bert model in bermodel.pkl file

In [32]:
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
reader = pipeline("question-answering", model=model_name, tokenizer=model_name)

with open('bertmodel.pkl', 'wb') as file:
    pickle.dump(reader, file)

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [34]:
vs_path= "/content/Output1.pkl"
bertmodel_path = "/content/bertmodel.pkl"
stmodel_path  = "/content/stmodel.pkl"
qsclient_path  = "/content/qs_client.pkl"



with open(bertmodel_path, 'rb') as file:
    bert = pickle.load(file)
with open(qsclient_path, 'rb') as file:
    qdrant_client = pickle.load(file)


with open(stmodel_path, 'rb') as file:
    st_encoder = pickle.load(file)

Making functions for finding the contexts that are closer to the query

In [35]:
collection_name = "BBProduct" # giving the name of the collection we made earlier
def closecontexts(question: str, top_k: int) -> list[str]:

    try:
        encoded_query = st_encoder.encode(question).tolist()
        result = qdrant_client.search(
            collection_name=collection_name,
            query_vector=encoded_query,
            limit=top_k,
        )  # search qdrant collection for context passage with the answer

        context = [
            [context.payload["product"], context.payload["story"]] for context in result
        ]
        return context
    except Exception as e:
        print({e})

Function to return the answer of the query (question asked)

In [36]:
def returnans(question: str, context: list[str]):

    results = []
    for c in context:
        answer = bert(question=question, context=c[1] )
        answer["product"] = c[0]
        results.append(answer)
        print()

    sorted_result = sorted(results, key=lambda x: x["score"], reverse=True)
    for i in range(len(sorted_result)):
        _out = sorted_result[i]["answer"]
        _prod = sorted_result[i]["product"]
        _sco = sorted_result[i]["score"]
#         print(f"{i+1}", end=" ")
        print(f"QUERY INPUT: {question}")
        print(f"OUTPUT: {_out} \nPREDICTION SCORE {_sco}\n\nReferred Product: {_prod}\n\n")
        return question,_out,_sco,_prod



List of query

In [37]:
queries = []
queries.append("suggest me one product for bathing")
queries.append("what is price of chia seeds")
queries.append("what is most loved hair product")
queries.append("suggest one  Anti- Bacterial Scrub Pad")
queries.append("price of Soan Papdi Special")
queries.append("what is most loved beauty product")

